# Learning constants in a symbolic regression task

One of the long standing "skeletons in the closet" of GP techniques is the constant finding problem. It is widely acknoledged that the "ephemeral random constant", de facto the main solution proposed to this problem, is far from being satisfactory


Using dCGP, we are here able to succesfully learn constants as well as expressions during evolution thanks to the hybridization of the evolutionary strategy with what may, essentially, be seen as a second order back-propagation algorithm

Lets first import dcgpy and pyaudi and set up things as to compute our CGP using the type "gdual" and thus get for free all derivatives

In [1]:
from dcgpy import expression_gdual_vdouble as expression
from dcgpy import kernel_set_gdual_vdouble as kernel_set
from pyaudi import gdual_vdouble as gdual
import pyaudi
from matplotlib import pyplot as plt
import numpy as np
from random import randint
%matplotlib inline

## 1 - The set of kernel functions

In [2]:
# note he use of the protected division "pdiv" (not necessary here)
# note the call operator (returns the list of kernels)
kernels = kernel_set(["sum", "mul", "diff","pdiv"])() 

## 2 - The ES-(1+$\lambda$) algorithm

In [28]:
def run_experiment(dCGP, offsprings, max_gen, x, yt, screen_output):
    # The offsprings chromosome, fitness and constant
    chromosome = [1] * offsprings
    fitness = [1] *offsprings
    constant = [1]*offsprings
    # Init the best as the initial random dCGP
    best_chromosome = dCGP.get()
    best_constant = 1.
    fit, _ = err2(dCGP, x, yt, best_constant)
    best_fitness = sum(fit.constant_cf)
    # Main loop over generations
    for g in range(max_gen):
        for i in range(offsprings):
            dCGP.set(best_chromosome)
            cumsum=0
            dCGP.mutate_active(i+1)
            fit, constant[i] = err2(dCGP, x, yt, best_constant)
            fitness[i] = sum(fit.constant_cf )
            chromosome[i] = dCGP.get()
        for i in range(offsprings):
            if fitness[i] <= best_fitness:
                if (fitness[i] != best_fitness):
                    best_chromosome = chromosome[i]
                    best_fitness = fitness[i]
                    best_constant = constant[i]
                    dCGP.set(best_chromosome)
                    if screen_output:
                        print("New best found: gen: ", g, " value: ", fitness[i],  dCGP.simplify(["x","c"]), "c =", best_constant)

        if best_fitness < 1e-7:
            break
    return g, best_chromosome, best_constant

## 3 - The test problems
As target functions, we define three different problems of increasing complexity:

P1: $x^5 - \pi x^3 + x$

P2: $x^5 - \pi x^3 + \frac{2\pi}x$

P3: $\frac{e x^5 + x^3}{x + 1}$

note how $\pi$ and $e$ are present in the expressions.

In [4]:
# The following functions create the target values for a gridded input x for different test problems
def data_P1(x):
    return x**5 - np.pi*x**3 + x
def data_P2(x):
    return x**5 - np.pi*x**3 + 2*np.pi / x
def data_P3(x):
    return (np.e*x**5 + x**3)/(x + 1)


## 4 - The error functions

In [5]:
# This is the quadratic error of a dCGP expression when the constant value is cin. The error is computed
# over the input points xin (of type gdual, order 0 as we are not interested in expanding the program w.r.t. these)
# The target values are contained in yt (of type gdual, order 0 as we are not interested in expanding the program w.r.t. these)
def err(dCGP, xin, yt, cin):
    c = gdual([cin], "c", 2)
    y = dCGP([xin,c])[0]
    return (y-yt)**2

# This is the quadratic error of the expression when the constant value is learned using a, one step, 
# second order method.
def err2(dCGP, xin, yt,cin):
    c = gdual([cin], "c", 2)
    y = dCGP([xin,c])[0]
    dc =  sum(err(dCGP,xin,yt,cin).get_derivative({"dc":1}))
    dc2 = sum(err(dCGP,xin,yt,cin).get_derivative({"dc":2}))
    if dc2 != 0:
        learned_constant = c - dc/dc2
        y = dCGP([xin, learned_constant])[0]
    else:
        learned_constant = c
    return (y-yt)**2, learned_constant.constant_cf[0]

##  Problem P1:  $x^5 - \pi x^3 + x$

In [30]:
x = np.linspace(1,3,10)
x = gdual(x)
yt = data_P1(x)

In [31]:
# We run nexp experiments and accumulate statistic for the ERT
nexp = 100
offsprings = 4
max_gen=1000
res = []
print("restart: \t gen: \t expression:")
for i in range(nexp):
    dCGP = expression(inputs=2, outputs=1, rows=1, cols=15, levels_back=16, arity=2, kernels=kernels, seed = randint(0,1233456))
    g, best_chromosome, best_constant = run_experiment(dCGP, offsprings,max_gen,x,yt, screen_output=False)
    res.append(g)
    dCGP.set(best_chromosome)
    if g < (max_gen-1):
        print(i, "\t\t", res[i], "\t", dCGP(["x","c"]), " a.k.a ", dCGP.simplify(["x","c"]), "c = ", best_constant)
res = np.array(res)

restart: 	 gen: 	 expression:
2 		 68 	 ['(x+(((x*x)*x)*(c+((x*x)+c))))']  a.k.a  [2*c*x**3 + x**5 + x] c =  -1.5707963267948968
3 		 413 	 ['((x*((x*x)*(c+(x*x))))+x)']  a.k.a  [c*x**3 + x**5 + x] c =  -3.1415926535897927
6 		 738 	 ['(x-(((c-(x*x))*x)*(x*x)))']  a.k.a  [-c*x**3 + x**5 + x] c =  3.141592653589793
7 		 245 	 ['(((((x*x)*(x*x))*(c+(x*x)))/x)+((x*x)/x))']  a.k.a  [c*x**3 + x**5 + x] c =  -3.1415926535897927
13 		 967 	 ['(x+(((x*x)*x)*(c+(x*x))))']  a.k.a  [c*x**3 + x**5 + x] c =  -3.1415926535897927
16 		 466 	 ['(((((x-x)+(x*x))/x)-((x*((x-x)+(x*x)))*(c-(x*x))))+(x*((x-x)+(x*x))))']  a.k.a  [-c*x**3 + x**5 + x**3 + x] c =  4.141592653589793
20 		 408 	 ['(((x*x)+(x/(x*c)))*(((x*x)+((c-c)+c))*x))']  a.k.a  [c*x**3 + x**5 + x + x**3/c] c =  -2.7821597820194426
30 		 747 	 ['((x*((x*x)*((c+c)+(x*x))))+x)']  a.k.a  [2*c*x**3 + x**5 + x] c =  -1.5707963267948963
42 		 402 	 ['(((x/x)*x)+((c+((x*((x/x)*x))+(x/x)))*((x*((x/x)*x))*x)))']  a.k.a  [c*x**3 + x**5 + x**3 + x] c = 

In [32]:
mean_gen = sum(res) / sum(res<(max_gen-1))
print("ERT Expected run time = avg. number of dCGP evaluations needed: ", mean_gen * offsprings)

ERT Expected run time = avg. number of dCGP evaluations needed:  15884.1818182


# 4 - Problem P2 - $x^5 - \pi x^3 + \frac{2\pi}x$


In [33]:
x = np.linspace(0.1,5,10) # we include points close to zero here to favour learning of 1/x
x = gdual(x)
yt = data_P2(x)


In [34]:
# We run nexp experiments and accumulate statistic for the ERT
nexp = 100
offsprings = 4
max_gen=5000
res = []
print("restart: \t gen: \t expression:")
for i in range(nexp):
    dCGP = expression(inputs=2, outputs=1, rows=1, cols=15, levels_back=16, arity=2, kernels=kernels, seed = randint(0,1233456))
    g, best_chromosome, best_constant = run_experiment(dCGP, offsprings,max_gen,x,yt, screen_output=False)
    res.append(g)
    dCGP.set(best_chromosome)
    if g < (max_gen-1):
        print(i, "\t\t", res[i], "\t", dCGP(["x","c"]), " a.k.a ", dCGP.simplify(["x","c"]), "c = ", best_constant)
res = np.array(res)

restart: 	 gen: 	 expression:
4 		 3788 	 ['((c/x)+(((c/(x*x))+(((x*x)-c)*(x*x)))*x))']  a.k.a  [-c*x**3 + 2*c/x + x**5] c =  3.141592653589791
9 		 4242 	 ['((((x*(x*x))*c)+((x*x)*(x*(x*x))))-((c+c)/x))']  a.k.a  [c*x**3 - 2*c/x + x**5] c =  -3.141592653589793
11 		 808 	 ['(((x*x)*(x*(x*x)))-((c/x)+((c/x)-((x*(x*x))*c))))']  a.k.a  [c*x**3 - 2*c/x + x**5] c =  -3.141592653589793
12 		 2388 	 ['(((((x*x)*(x*x))+(c/(x*x)))+((c/(x*x))-(c*(x*x))))*x)']  a.k.a  [-c*x**3 + 2*c/x + x**5] c =  3.141592653589793
19 		 3782 	 ['(((((x*x)*(c+(x*x)))*(x*x))-(c+c))/x)']  a.k.a  [c*x**3 - 2*c/x + x**5] c =  -3.141592653589794
25 		 1475 	 ['(((x*(x*x))*((x*x)-c))+((c/x)+(c/x)))']  a.k.a  [-c*x**3 + 2*c/x + x**5] c =  3.1415926535897944
27 		 3839 	 ['((((c/x)-(((x*x)-c)-((((x*x)-c)*x)*(x*x))))+((x*x)-c))+(c/x))']  a.k.a  [-c*x**3 + 2*c/x + x**5] c =  3.1415926535897896
29 		 964 	 ['((((x*x)-c)*((x*x)*x))+((c/x)+(c/x)))']  a.k.a  [-c*x**3 + 2*c/x + x**5] c =  3.141592653589794
34 		 3963 	 ['(((c+

In [35]:
mean_gen = sum(res) / sum(res<(max_gen-1))
print("ERT Expected run time = avg. number of dCGP evaluations needed: ", mean_gen * offsprings)

ERT Expected run time = avg. number of dCGP evaluations needed:  73515.8333333


# 4 - Problem P3 - $\frac{e x^5 + x^3}{x + 1}$


In [41]:
x = np.linspace(-0.9,1,10)
x = gdual(x)
yt = data_P3(x)


In [42]:
# We run nexp experiments and accumulate statistic for the ERT
nexp = 100
offsprings = 4
max_gen=5000
res = []
print("restart: \t gen: \t expression:")
for i in range(nexp):
    dCGP = expression(inputs=2, outputs=1, rows=1, cols=15, levels_back=16, arity=2, kernels=kernels, seed = randint(0,1233456))
    g, best_chromosome, best_constant = run_experiment(dCGP, offsprings,max_gen,x,yt, screen_output=False)
    res.append(g)
    dCGP.set(best_chromosome)
    if g < (max_gen-1):
        print(i, "\t\t", res[i], "\t", dCGP(["x","c"]), " a.k.a ", dCGP.simplify(["x","c"]), "c = ", best_constant)
res = np.array(res)

restart: 	 gen: 	 expression:
2 		 4681 	 ['((((x*x)*(x*x))*(((c+c)*(x*x))+((x*x)/(x*x))))/((x*x)+x))']  a.k.a  [2*c*x**6/(x**2 + x) + x**4/(x**2 + x)] c =  1.359140914229522
15 		 847 	 ['(((x*x)*x)*((x/((x*x)+x))+((x/((x*x)+x))*((x*x)*(c+c)))))']  a.k.a  [2*c*x**6/(x**2 + x) + x**4/(x**2 + x)] c =  1.3591409142295228
27 		 3606 	 ['((((c*(x*x))+(x/x))*(x*x))/((x/(x*x))+(x/x)))']  a.k.a  [c*x**4/(1 + 1/x) + x**2/(1 + 1/x)] c =  2.7182818284590486
30 		 3211 	 ['(((c*x)*x)*(((((c*x)+(c*x))*((c*x)*x))+x)/(c+(c*x))))']  a.k.a  [2*c**3*x**5/(c*x + c) + c*x**3/(c*x + c)] c =  1.165825292296718
38 		 783 	 ['((x*(c*x))*((x-((c*x)*(x*x)))/(c+(c*x))))']  a.k.a  [-c**2*x**5/(c*x + c) + c*x**3/(c*x + c)] c =  -2.7182818284590367
52 		 2651 	 ['((x*x)*(((x-(x*x))*((((x*x)*c)*c)/(((x*x)*c)*c)))-((((x*x)*c)*(x*x))/(x+(x*x)))))']  a.k.a  [-c*x**6/(x**2 + x) - x**4 + x**3] c =  -3.718281828459044
65 		 459 	 ['((((c*x)*(x*x))*x)-(((c*x)*(((c*x)*(x*x))/(c+(c*x))))-(((c*x)*(x*x))/(c+(c*x)))))']  a.k.a

In [43]:
mean_gen = sum(res) / sum(res<(max_gen-1))
print("ERT Expected run time = avg. number of dCGP evaluations needed: ", mean_gen * offsprings)

ERT Expected run time = avg. number of dCGP evaluations needed:  133358.857143
